# Classifing handwritten digits with Convolutional Neural Networks
### with Tensorflow
useful links:
- https://www.tensorflow.org/versions/r0.11/tutorials/deep_cnn/index.html
- https://martin-thoma.com/tensor-flow-quick/

Below is some generic Code for setting up an interactive tensorflow session and the mnist dataset.

In [2]:
import tensorflow as tf
from functools import *
import time
from tensorflow.examples.tutorials.mnist import input_data
#onehot means a vector is 1 in a single dimension and 0 in every other
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Model

In [86]:
n_inputs = 784
n_classes = 10
batch_size = 50

x = tf.placeholder('float', [None, n_inputs])
y = tf.placeholder('float',[None, 10])


weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [16]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [75]:
def conv2d(x, W, b, k=1):
    x = tf.nn.conv2d(x, W, strides=[1, k, k, 1] , padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [87]:
def conv_neural_net(x, weights, biases):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    
    # Pooling 1
    conv1 = maxpool2d(conv1, k=2)

    # Convolution 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    
    # Pooling 2
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

    return out

In [88]:
def train_and_test_conv_neural_net(x, y, weights, biases, epochs = 1):
    
    pred = conv_neural_net(x, weights, biases)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred,y))
    opt = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for e in range(epochs):
            epoch_loss = 0
            for i in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([opt, cost], 
                                feed_dict = {x:epoch_x, y:epoch_y})
                epoch_loss = epoch_loss + c
            print('Epoch ' + str(e+1) + ' out of ' + 
            str(epochs) + ' / Loss: ' + str(epoch_loss))
            
            correct = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print 'Accuracy: ' + str(accuracy.eval({x:mnist.test.images, 
                                                y:mnist.test.labels}))

        print 'Training finished!'  

## Training

In [89]:
start_time = time.time()
train_and_test_conv_neural_net(x, y, weights, biases, 15)
print 'Time: ' +  str(time.time() - start_time)

Epoch 1 out of 15 / Loss: 2048269.96789
Accuracy: 0.9521
Epoch 2 out of 15 / Loss: 416296.553819
Accuracy: 0.9562
Epoch 3 out of 15 / Loss: 230842.755202
Accuracy: 0.9515
Epoch 4 out of 15 / Loss: 149993.567395
Accuracy: 0.9741
Epoch 5 out of 15 / Loss: 104185.315256
Accuracy: 0.9787
Epoch 6 out of 15 / Loss: 65479.1788944
Accuracy: 0.9785
Epoch 7 out of 15 / Loss: 56342.0501099
Accuracy: 0.9819
Epoch 8 out of 15 / Loss: 37191.8697067
Accuracy: 0.9757
Epoch 9 out of 15 / Loss: 31626.6323467
Accuracy: 0.9802
Epoch 10 out of 15 / Loss: 26357.9734757
Accuracy: 0.9839
Epoch 11 out of 15 / Loss: 19355.0131723
Accuracy: 0.9817
Epoch 12 out of 15 / Loss: 17152.4177485
Accuracy: 0.9849
Epoch 13 out of 15 / Loss: 18811.924521
Accuracy: 0.9827
Epoch 14 out of 15 / Loss: 11879.7064547
Accuracy: 0.9848
Epoch 15 out of 15 / Loss: 13809.3100464
Accuracy: 0.9799
Training finished!
Time: 2197.68839502


## Results

### CNN 1

In [85]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 48])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 48, 48])),
    'wc3': tf.Variable(tf.random_normal([5, 5, 48, 32])),
    'wd1': tf.Variable(tf.random_normal([7*7*32, 1024])),
    'wd2': tf.Variable(tf.random_normal([1024,512])),
    'out': tf.Variable(tf.random_normal([512, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([48])),
    'bc2': tf.Variable(tf.random_normal([48])),
    'bc3': tf.Variable(tf.random_normal([32])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'bd2': tf.Variable(tf.random_normal([512])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


def conv_neural_net(x, weights, biases):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], k=1)
    
    # Pooling 1
    conv1 = maxpool2d(conv1, k=2)

    # Convolution 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], k=1)
    
    # Pooling 2
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution 2
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], k=1)
    
    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)

    # Predictions
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])

    return out

Epoch 1 out of 15 / Loss: 468610514.509
Accuracy: 0.9451

Epoch 2 out of 15 / Loss: 95684529.7457
Accuracy: 0.9519

Epoch 3 out of 15 / Loss: 47215840.6904
Accuracy: 0.9607

Epoch 4 out of 15 / Loss: 32338930.3717
Accuracy: 0.9625

Epoch 5 out of 15 / Loss: 23484511.0364
Accuracy: 0.9688

Epoch 6 out of 15 / Loss: 17932300.7021
Accuracy: 0.9704

Epoch 7 out of 15 / Loss: 13503283.2412
Accuracy: 0.9712

Epoch 8 out of 15 / Loss: 10075762.2565
Accuracy: 0.974

Epoch 9 out of 15 / Loss: 8688486.11646
Accuracy: 0.9795

Epoch 10 out of 15 / Loss: 7702904.45678
Accuracy: 0.9766

Epoch 11 out of 15 / Loss: 7166355.99678
Accuracy: 0.9769

Epoch 12 out of 15 / Loss: 5661994.87721
Accuracy: 0.9767

Epoch 13 out of 15 / Loss: 5315591.25652
Accuracy: 0.9794

Epoch 14 out of 15 / Loss: 4978734.87103
Accuracy: 0.982

Epoch 15 out of 15 / Loss: 4567343.04259
Accuracy: 0.9764

Training finished!
Time: 3057.45236683

## CNN 2

In [90]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def conv_neural_net(x, weights, biases):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    
    # Pooling 1
    conv1 = maxpool2d(conv1, k=2)

    # Convolution 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    
    # Pooling 2
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

    return out

Epoch 1 out of 15 / Loss: 2048269.96789
Accuracy: 0.9521

Epoch 2 out of 15 / Loss: 416296.553819
Accuracy: 0.9562

Epoch 3 out of 15 / Loss: 230842.755202
Accuracy: 0.9515

Epoch 4 out of 15 / Loss: 149993.567395
Accuracy: 0.9741

Epoch 5 out of 15 / Loss: 104185.315256
Accuracy: 0.9787

Epoch 6 out of 15 / Loss: 65479.1788944
Accuracy: 0.9785

Epoch 7 out of 15 / Loss: 56342.0501099
Accuracy: 0.9819

Epoch 8 out of 15 / Loss: 37191.8697067
Accuracy: 0.9757

Epoch 9 out of 15 / Loss: 31626.6323467
Accuracy: 0.9802

Epoch 10 out of 15 / Loss: 26357.9734757
Accuracy: 0.9839

Epoch 11 out of 15 / Loss: 19355.0131723
Accuracy: 0.9817

Epoch 12 out of 15 / Loss: 17152.4177485
Accuracy: 0.9849

Epoch 13 out of 15 / Loss: 18811.924521
Accuracy: 0.9827

Epoch 14 out of 15 / Loss: 11879.7064547
Accuracy: 0.9848

Epoch 15 out of 15 / Loss: 13809.3100464
Accuracy: 0.9799

Training finished!
Time: 2197.68839502